In [23]:
pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached multitasking-0.0.12.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached frozendict-2.4.6-py311-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.2.tar.gz (949 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached curl_cffi-0.13.0.tar.gz (151 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 175, in prepare_metadata_for_build_wheel
          return hook(

Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached multitasking-0.0.12.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached frozendict-2.4.6-py311-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.2.tar.gz (949 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached curl_cffi-0.13.0.tar.gz (151 kB)
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/var/mobile/Containers/Data/Application/EC77ADF6-EFDD-452A-A99F-812D26B10801/Library/lib/python3.11/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 175, in prepare_metadata_for_build_wheel
          return hook(

In [ ]:
pip install matplotlib

In [10]:
#import yfinance as yf
import yahoofinancials as yf
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Define custom scoring function
def score_func(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def predeccionBolsa(ticker_, startTime):
    data = yf.download(ticker_, start=startTime)
    #Extraemos la columna 'Close' del DataFrame 'data' y lo almacenamos en un nuevo DataFrame llamado 'data_close'
    data_close = data[['Close']]
    #Restablecemos el índice del DataFrame 'data_close' para garantizar un índice secuencial.
    data_close.reset_index(inplace=True)
    #Cambiamos el nombre de las columnas del DataFrame 'data_close' a 'ds' e 'y', que representan la fecha y el precio de cierre
    data_close.columns = ['ds', 'y']
    #Calculamos el promedio móvil (MA) de 200 días del precio de cierre utilizando el método 'rolling' y asignamos los valores 
    #a una nueva columna 'MA' en el DataFrame 'data_close'. Cualquier valor faltante resultante de la operación de balanceo 
    #se completa con cero.
    data_close.loc[:, 'MA'] = data_close['y'].rolling(window=200).mean().fillna(0)
    #Agregamos una columna 'Indicator' al marco de datos 'data_close', que indica si "Buy" o "Sell" en función de una 
    #comparación entre el precio de cierre ('y') y el promedio móvil ('MA').
    #Si el precio de cierre es mayor que la media móvil, se marca como "Buy"; de lo contrario, se marca como "Sell".
    data_close.loc[:, 'Indicator'] = np.where(data_close['y'] > data_close['MA'], 'Buy', 'Sell')

    #Inicializamos el objeto TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=5)
    #Inicializamos una lista vacía para almacenar las puntuaciones.
    scores = []

    
    for train_index, test_index in tscv.split(data_close):
        #Obtenemos los datos de entrenamiento y test para esta división
        train_data = data_close.iloc[train_index]
        test_data = data_close.iloc[test_index]
        #Ajustamos el modelo a los datos de entrenamiento.
        m = Prophet(yearly_seasonality=True)
        m.fit(train_data)
        #Hacemos predicciones sobre los datos de prueba.
        future = m.make_future_dataframe(periods=365*5)
        forecast = m.predict(future)
        test_predictions = forecast.iloc[-len(test_data):][['yhat']]
        #Calculamos la puntuación de esta división.
        score = score_func(test_data[['y']], test_predictions)
        scores.append(score)
    mean_score = sum(scores) / len(scores)

    #Ajustamos el modelo con todos los datos.
    m = Prophet(yearly_seasonality=True)
    m.fit(data_close)

    #Hacemos las predicciones
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    return forecast, mean_score, m

def graficaPredicciones(forecast, mean_score,mm):
    # Plot the forecast
    plt.figure(figsize=(15, 8))
    fig1 = mm.plot(forecast)
    # Add labels and a title to the graph
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title('Stock Price with Buy and Sell Indicators')
    # Add gridlines to the graph
    plt.grid(True)
    # Print the mean score
    print("Cross Validation Score : ", mean_score)
    # Show the plot
    plt.show()

ModuleNotFoundError: No module named 'yahoofinancials'

In [ ]:
ticker = 'SPY'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
forecast[forecast['ds']=='2024-09-03']
forecast[forecast['ds']>'2024-08-01']
forecast.to_csv('SPY.csv', index=False)

In [ ]:
forecast[forecast['ds']=='2024-07-15']

In [ ]:
ticker = 'WCBR'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'SWDA.MI'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'XAIX.DE'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'VGT'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'BITO'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'SCHG'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
forecast.to_csv('SCHG.csv', index=False)

In [ ]:
ticker = 'IVV'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
forecast.to_csv('IVV.csv', index=False)

In [ ]:
ticker = 'VWRL.AS'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'BITO'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)

In [ ]:
ticker = 'WCBR'
startTime='2021-01-01'
forecast, mean_score,mm=predeccionBolsa(ticker, startTime)

graficaPredicciones(forecast, mean_score,mm)